In [10]:
import subprocess
import os
import boto3
from botocore.exceptions import NoCredentialsError

def download_and_convert_stream(url, quality, filename, bucket_name, s3_path):
    # Define file paths
    temp_file = f"{filename}.ts"
    output_file = f"{filename}.mp4"

    try:
        # Step 1: Download video stream with Streamlink
        streamlink_command = ["streamlink", url, quality, "-o", temp_file]
        print(f"Running Streamlink command: {' '.join(streamlink_command)}")

        # Run Streamlink with an extended timeout or no timeout
        streamlink_process = subprocess.run(streamlink_command, check=True, timeout=3600)
        
        # Step 2: Convert .ts file to .mp4 and trim to 10 minutes with FFmpeg
        ffmpeg_command = ["ffmpeg", "-ss", "300", "-i", temp_file, "-t", "600", "-c", "copy", output_file]
        print(f"Running FFmpeg command: {' '.join(ffmpeg_command)}")
        
        # Run FFmpeg with an extended timeout or no timeout
        ffmpeg_process = subprocess.run(ffmpeg_command, check=True, timeout=3600)
        
        print(f"Video saved as {output_file}")

        # Step 3: Upload to S3
        s3 = boto3.client('s3')
        try:
            s3.upload_file(output_file, bucket_name, s3_path)
            print(f"File uploaded to s3://{bucket_name}/{s3_path}")
        except NoCredentialsError:
            print("AWS credentials not available.")

    except subprocess.TimeoutExpired:
        print("The process timed out.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
    finally:
        # Cleanup: Remove the temporary .ts and .mp4 files
        if os.path.exists(temp_file):
            os.remove(temp_file)
        if os.path.exists(output_file):
            os.remove(output_file)

# Example usage:
videos = [
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGODdmYjI0MTQ1M2FlNDBiMjhmYmM4YWMzY2RlMWZjZTAlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTUyNjA5NHx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDR1hKNDFOVFRNR1pWUldCR1BaM1o2RV9QUk98dG9rZW46ZXRkTDd1clU1dFVLWDAtMWpwZ29MQS1DbjdWbEluLS1WM1FPTkdXaWM4QV8_/c8059270529e63e492b34317933c3dca/out/v1/87fb241453ae40b28fbc8ac3cde1fce0/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "best",
        "name": "main_broadcast"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGMmNhNmU1N2ExMjMyNDc2ZGI2MjIzYjhlZDcwYTM0ZTklMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTMzMXx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtROFRQSzhRTVYzTllCMTJKWVk1Wl9QUk98dG9rZW46QkFoQXlnZFBjSnNFLVlxaEkxS1ZCSVE4ZEdidGo4S2ZOcWdvMzVEMGpid18_/c8059270529e63e492b34317933c3dca/out/v1/2ca6e57a1232476db6223b8ed70a34e9/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "best",
        "name": "tracker"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGZTdlZjQzZDE1OTQ2NDQ0MGI1MDc2OGI4YzBkODUwZGUlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTM1M3x0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtRWUQ2RzRaN0pXRkQxMjVXV1I5Sl9QUk98dG9rZW46UTBNRVZ4cXZVZkg4c29KSWg2YnVpVy1ZNWFObWdrN2ZPUEVoRElzRlpPa18_/c8059270529e63e492b34317933c3dca/out/v1/e7ef43d159464440b50768b8c0d850de/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "best",
        "name": "data_feed"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGMWEwNmZjOGJlMWE1NDRjMmFhNmM0ZjgyOTIyOTEyOTAlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTM4MXx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtSUzhUQTJDVFlDQ1oyREtYNlkzR19QUk98dG9rZW46VnNZYlRXM2ZtUmpyMm5TUn45Z2Qzd3lPUkdhVVdiRE1RWjFQMWw4Q2ZJc18_/c8059270529e63e492b34317933c3dca/out/v1/1a06fc8be1a544c2aa6c4f8292291290/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_VER"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGODlhZmI3ZDM0MGRmNDg1NzlhZWYyMjA5MTAyOTMzNWMlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTQxN3x0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtTV0tBMjg4Nlg3OFZIUjNYSjVDSF9QUk98dG9rZW46U0lIaENqbUthYzdhRXoxZ3lpeDItbmpQcTZvS3lJbDNMUjhNOU1LQXcwTV8_/c8059270529e63e492b34317933c3dca/out/v1/89afb7d340df48579aef22091029335c/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_SAR"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGMjU2YmU3OTBmMWQxNDA1MGE4ODAyMWRmNjYyZWU1ZjIlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTQ2N3x0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtWQ1lOQ1YxQVEyVkdXQ01TQ1FFQ19QUk98dG9rZW46dHZHaXdwMUlLMEhkfnNFOGwyQ01IZW5tbElJdlc4U3g3cW4xZkhRQTZoRV8_/c8059270529e63e492b34317933c3dca/out/v1/256be790f1d14050a88021df662ee5f2/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_RIC"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGNzA0MjE0OGVkMjQxNGM1ODk3MjRjZTQ1MDkyMGRiNDklMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTQ5OXx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtXQ1FQSlFIM0cyVFZLMDdLMjBQUF9QUk98dG9rZW46T3BkckpDUk1WbkxRSEp3UzQ0SGNKYzZkQm5SNklBV1hHOUZVVzNuZlhPRV8_/c8059270529e63e492b34317933c3dca/out/v1/7042148ed2414c589724ce450920db49/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_NOR"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGODk2MjA3NTVhOTkyNDk1MTk2ZmRlY2MyNGQzMDU3NjAlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTUyMnx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtYM0FNUDY0UkVZRzhUMlZRWjQxQl9QUk98dG9rZW46TjlBZ29ubUV2SGh1Z0RPbDZMeWZFd0xKTk9OMTNWZmdyazZGaU9mTHpNQV8_/c8059270529e63e492b34317933c3dca/out/v1/89620755a992495196fdecc24d305760/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_GAS"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGMzhmYWUwOTE0YWIxNDc1MThjNjkyMDZlMWQyZDA3YjUlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTU2MHx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtZODdLRDY4UEtNS1dUV0gzMEZKWl9QUk98dG9rZW46WkxvM05MZnNKUVgtVkZaOWUwMUl6N1RpR3lpNTRiMkNIRnV1bktkc0V3MF8_/c8059270529e63e492b34317933c3dca/out/v1/38fae0914ab147518c69206e1d2d07b5/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_PER"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGNzYyZTI5N2FlYWE3NDJiMThmNzlkZjc1ZmJjMzAwZGMlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTU4NXx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtaMFFTS0dFTVkzMFNaTlJUUVAzSl9QUk98dG9rZW46dmg3Z1BtRWxSfjRpRXBmeXY1Z1U5QTVHMGs3WnNFa005SkhUUG4yMXVYWV8_/c8059270529e63e492b34317933c3dca/out/v1/762e297aeaa742b18f79df75fbc300dc/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_ALO"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGYTM3MTAxZTYzNTU0NDQzMmIyOWEwZDhiMGRmNzU4ODMlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTYwOHx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtaUFhXTU5TMTVYWVJKVEdBNEU0RF9QUk98dG9rZW46ZGt3Q0NMa2JUZjBJYUxUTDRNbE9NcTZHOWJ0bkxlT21YVGRUdFFmaDJ1UV8_/c8059270529e63e492b34317933c3dca/out/v1/a37101e635544432b29a0d8b0df75883/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_LEC"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGNTk5NDNiNDVmMzRhNDEyYjliMzE2NDM5MjZmYTk3NjAlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTYzNnx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE0wSFpERjY2QzI5Q1A2WVpHRzRLNl9QUk98dG9rZW46Ri1aVXhndlh1VGRlTzhnVW82NEVpVzM5dkNkcGlBLTMtN3Q2dkI4UlVSWV8_/c8059270529e63e492b34317933c3dca/out/v1/59943b45f34a412b9b31643926fa9760/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_STR"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGMzllNzNhY2M5Y2FkNDEzNmFiMTI2MWZhNTQ3MTVhNjclMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTczN3x0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE0zTVRTUk1NU0dOWThZQTBQVkdKNV9QUk98dG9rZW46bWxJdHVHM1VVcmV0dUUxSmFjRVFZYm5FT3hjb3hVRGJOeWZscjF4RlhMMF8_/c8059270529e63e492b34317933c3dca/out/v1/39e73acc9cad4136ab1261fa54715a67/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_MAG"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGZDU1ZTIxM2ViNTM3NGI4MGI5MmRjYmMzODE5NjA5YzIlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTc3Nnx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE00VkpUV1pBN0oxNERQOTg0MUNIVl9QUk98dG9rZW46YkFQdHZZMkxBZ0FqUWpwS2M4Sy1vNnpxdHBjRXFvbGptOXpqdmIzT1l3d18_/c8059270529e63e492b34317933c3dca/out/v1/d55e213eb5374b80b92dcbc3819609c2/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_TSU"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGNDI0OTZlNDc1NmI2NGNmNDk3ZTBhMzgxNzBmMjRkNzAlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTgxM3x0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE01WjMxODdWVFFDU1A5ODFaU0U3Ql9QUk98dG9rZW46OTcyWHBNNFlOaDBtZWRQMVNLMDg2bm9JYTVROFRERUlYYnRSLVZtbEFDSV8_/c8059270529e63e492b34317933c3dca/out/v1/42496e4756b64cf497e0a38170f24d70/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_ALB"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGMzg5ZWZkYzE0ODUwNGRhNmE2YWQ1OGRlMWEzYTFjMzYlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTgzN3x0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE02UFY0NkhSNlhYWkNGWjAzTk5LQ19QUk98dG9rZW46RXZ3NXpGRTgwcUFrM1JZVi0tcW1oR1lWTHlwSTdjWHNSRm9ZcExofnkyY18_/c8059270529e63e492b34317933c3dca/out/v1/389efdc148504da6a6ad58de1a3a1c36/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_ZHO"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGNWRmZjgzMzU0MDg2NDE0ZjgwMTAxZjM4ZDJhN2M5ZmYlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTg2NHx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE03SDBERTNHQ1pEMVJWNVI3QVI5Sl9QUk98dG9rZW46bzhFUmJ0cU9hY3pYOEY5bHkzfndMbGJaTDFIWUFSaVFBQ3F0NFVxYUx3WV8_/c8059270529e63e492b34317933c3dca/out/v1/5dff83354086414f80101f38d2a7c9ff/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_HUL"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGOWNlZmQ5NzNlNThhNDJjM2EyM2QwZDg2MDdiNTdhNDElMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTg5N3x0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE04SDFQSkNKR0Y4WlM5MUc0S0JGWF9QUk98dG9rZW46bk1oMTF4eVZZWnBDODJDY0szRXJLcGp0TWFDV3l-c3ZpWnZOazZqaWVYUV8_/c8059270529e63e492b34317933c3dca/out/v1/9cefd973e58a42c3a23d0d8607b57a41/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_OCO"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGZGY1ZTM2OTZiZjJlNDg1NmE4OWJmNTk2OWM0YjM3YTIlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU1MDExMXx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE1GMkJBR0RTVlA5UDJTMjdZWThaTV9QUk98dG9rZW46Y1h2Q29Xamo5R2UxMmJsaDAxOGhBYW1FU2RyQ1VhQ1FHdXhpemk5MFZqRV8_/c8059270529e63e492b34317933c3dca/out/v1/df5e3696bf2e4856a89bf5969c4b37a2/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_HAM"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGYzRlMzYyZTRiZWZjNDU0Yjk4YTYzNTg0MzZkMmNlYTclMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU1MDEzNHx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE1GUkNBS0hRUktGNFdUQjBNS0FUWF9QUk98dG9rZW46aU9UNlFvSWJIeEt2SE5NUU5GOXI4QzJaUFRxcHlPVEktWlowbDNlZjBjd18_/c8059270529e63e492b34317933c3dca/out/v1/c4e362e4befc454b98a6358436d2cea7/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_SAI"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGMTUyOTYzZjM5MjJkNGVmZjhlZWUxNTBlNDkyZjMwNzYlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU1MDE2NXx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE1HUFA1Tk1aNzlQNjdGQ1lONDJNTV9QUk98dG9rZW46cEZYWXk5VUJDaVdCVX5XaS15R3ZHSzdkTUJjSnVpcTNid1p0Z0xzRHVYQV8_/c8059270529e63e492b34317933c3dca/out/v1/152963f3922d4eff8eee150e492f3076/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_RUS"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGOTJlOGZiYzk5YzkwNDJiMGJkMTZiNTc1YjQ4N2U4ZjUlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU1MDE5Nnx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE1ITjZaTU5IQzFGUjJWQ1c4QjJGU19QUk98dG9rZW46WnU0bkVDdFQ5eFZaN2hBODJkYn55OTFvblQ2WVlRU1N0Q3hQfkg3Nzc4OF8_/c8059270529e63e492b34317933c3dca/out/v1/92e8fbc99c9042b0bd16b575b487e8f5/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_BOT"
    },
    {
        "url": "https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGZmI1OTg0OGM5NDc4NDIyY2EzYjE3ZDA4ZmFiNDJmY2ElMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU1MDIyNXx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE1KSEE5S0pYMlNOOFFRTTJSVjRFTl9QUk98dG9rZW46VTVPfkU2YWM1RE9QTFFkUUhRbXlKVW04cjlwYm1GUUJVd1drZW1hZWJmZ18_/c8059270529e63e492b34317933c3dca/out/v1/fb59848c9478422ca3b17d08fab42fca/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8",
        "quality": "worst",
        "name": "driver_BOT"
    },

]

bucket_name = "f1-race-videos"
race = "abu_dhabi"
for video in videos: 
    s3_path = f"{race}/{video['name']}.mp4"
    download_and_convert_stream(video['url'], video['quality'], video['name'], bucket_name, s3_path)



Running Streamlink command: streamlink https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGODdmYjI0MTQ1M2FlNDBiMjhmYmM4YWMzY2RlMWZjZTAlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTUyNjA5NHx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDR1hKNDFOVFRNR1pWUldCR1BaM1o2RV9QUk98dG9rZW46ZXRkTDd1clU1dFVLWDAtMWpwZ29MQS1DbjdWbEluLS1WM1FPTkdXaWM4QV8_/c8059270529e63e492b34317933c3dca/out/v1/87fb241453ae40b28fbc8ac3cde1fce0/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8 best -o main_broadcast.ts
[cli][info] Found matching plugin hls for URL https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGODdmYjI0MTQ1M2FlNDBiMjhmYmM4YWMzY2RlMWZjZTAlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTUyNjA5NHx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDR1hKNDFOVF

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --ena

Video saved as main_broadcast.mp4
File uploaded to s3://f1-race-videos/abu_dhabi/main_broadcast.mp4
Running Streamlink command: streamlink https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGMmNhNmU1N2ExMjMyNDc2ZGI2MjIzYjhlZDcwYTM0ZTklMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTMzMXx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtROFRQSzhRTVYzTllCMTJKWVk1Wl9QUk98dG9rZW46QkFoQXlnZFBjSnNFLVlxaEkxS1ZCSVE4ZEdidGo4S2ZOcWdvMzVEMGpid18_/c8059270529e63e492b34317933c3dca/out/v1/2ca6e57a1232476db6223b8ed70a34e9/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8 best -o tracker.ts
[cli][info] Found matching plugin hls for URL https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGMmNhNmU1N2ExMjMyNDc2ZGI2MjIzYjhlZDcwYTM0ZTklMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczM

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --ena

Video saved as tracker.mp4
File uploaded to s3://f1-race-videos/abu_dhabi/tracker.mp4
Running Streamlink command: streamlink https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGZTdlZjQzZDE1OTQ2NDQ0MGI1MDc2OGI4YzBkODUwZGUlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTM1M3x0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSEtRWUQ2RzRaN0pXRkQxMjVXV1I5Sl9QUk98dG9rZW46UTBNRVZ4cXZVZkg4c29KSWg2YnVpVy1ZNWFObWdrN2ZPUEVoRElzRlpPa18_/c8059270529e63e492b34317933c3dca/out/v1/e7ef43d159464440b50768b8c0d850de/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8 best -o data_feed.ts
[cli][info] Found matching plugin hls for URL https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGZTdlZjQzZDE1OTQ2NDQ0MGI1MDc2OGI4YzBkODUwZGUlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU0OTM1M3x0d

...

Video saved as driver_RUS.mp4
File uploaded to s3://f1-race-videos/abu_dhabi/driver_RUS.mp4
Running Streamlink command: streamlink https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGOTJlOGZiYzk5YzkwNDJiMGJkMTZiNTc1YjQ4N2U4ZjUlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU1MDE5Nnx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE1ITjZaTU5IQzFGUjJWQ1c4QjJGU19QUk98dG9rZW46WnU0bkVDdFQ5eFZaN2hBODJkYn55OTFvblQ2WVlRU1N0Q3hQfkg3Nzc4OF8_/c8059270529e63e492b34317933c3dca/out/v1/92e8fbc99c9042b0bd16b575b487e8f5/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8 worst -o driver_BOT.ts
[cli][info] Found matching plugin hls for URL https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGOTJlOGZiYzk5YzkwNDJiMGJkMTZiNTc1YjQ4N2U4ZjUlMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU1M

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --ena

Video saved as driver_BOT.mp4
File uploaded to s3://f1-race-videos/abu_dhabi/driver_BOT.mp4
Running Streamlink command: streamlink https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGZmI1OTg0OGM5NDc4NDIyY2EzYjE3ZDA4ZmFiNDJmY2ElMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU1MDIyNXx0dGw6MTQ0MHxnZW86VVN8c3RyZWFtVHlwZTpITFN8c2Vzc2lvbklkOjIxMzY3NDIyMV8wMUpDSE1KSEE5S0pYMlNOOFFRTTJSVjRFTl9QUk98dG9rZW46VTVPfkU2YWM1RE9QTFFkUUhRbXlKVW04cjlwYm1GUUJVd1drZW1hZWJmZ18_/c8059270529e63e492b34317933c3dca/out/v1/fb59848c9478422ca3b17d08fab42fca/33957307c5b848a6bea693bb400a7166/e4d49ef8077f4032a097e2b0c8a962bb/index.m3u8 worst -o driver_BOT.ts
[cli][info] Found matching plugin hls for URL https://ott-video-fer-cf.formula1.com/v2/pa_cGF0aDolMkZjODA1OTI3MDUyOWU2M2U0OTJiMzQzMTc5MzNjM2RjYSUyRm91dCUyRnYxJTJGZmI1OTg0OGM5NDc4NDIyY2EzYjE3ZDA4ZmFiNDJmY2ElMkYzMzk1NzMwN2M1Yjg0OGE2YmVhNjkzYmI0MDBhNzE2NnxraWQ6MTA0MnxleHA6MTczMTU1M

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --ena

Video saved as driver_BOT.mp4
File uploaded to s3://f1-race-videos/abu_dhabi/driver_BOT.mp4


In [1]:
import os
import boto3
from collections import defaultdict

# Specify your bucket name
bucket_name = 'f1-race-videos'
s3 = boto3.client('s3')

# Organize files by "folders"
files_by_folder = defaultdict(list)
for obj in s3.list_objects_v2(Bucket=bucket_name)['Contents']:
    file_key = obj['Key']
    parts = file_key.split('/')
    
    if len(parts) > 1:
        folder = parts[0]
        file_name = '/'.join(parts[1:])
        files_by_folder[folder].append({'key': file_key, 'name': file_name, 'size': obj['Size']})
    else:
        files_by_folder['root'].append({'key': file_key, 'name': file_key, 'size': obj['Size']})

# HTML template for file list
def generate_html(folder_name, files):
    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>File Listing - {folder_name}</title>
        <style>
            body {{ font-family: Arial, sans-serif; background-color: #f4f6f8; margin: 0; padding: 0; color: #333; }}
            .container {{ max-width: 800px; margin: 50px auto; background: #fff; padding: 20px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); }}
            h1 {{ font-size: 24px; text-align: center; color: #555; }}
            .file-list, .folder-list {{ list-style: none; padding: 0; margin: 20px 0 0; }}
            .item {{ display: flex; align-items: center; padding: 12px 16px; border-bottom: 1px solid #e0e0e0; transition: background-color 0.2s; }}
            .item:hover {{ background-color: #f9f9f9; }}
            .icon {{ font-size: 20px; color: #3c91e6; margin-right: 12px; }}
            .name {{ color: #555; font-size: 16px; text-decoration: none; flex-grow: 1; }}
            .name:hover {{ text-decoration: underline; }}
            .size {{ font-size: 14px; color: #888; }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>{folder_name}</h1>
            <ul class="folder-list">
    """
    
    # Add folders on the main page only
    if folder_name == "root":
        for folder in files_by_folder.keys():
            if folder != "root":
                html_content += f"""
                <li class="item">
                    <span class="icon">📁</span>
                    <a href="{folder}/index.html" class="name">{folder}</a>
                </li>
                """

    # Add files for this folder
    html_content += "</ul><ul class='file-list'>"
    for file in files:
        file_url = f"https://{bucket_name}.s3.amazonaws.com/{file['key']}"
        file_size_mb = round(file['size'] / (1024 * 1024), 2)
        html_content += f"""
        <li class="item">
            <span class="icon">📄</span>
            <a href="{file_url}" class="name">{file['name']}</a>
            <span class="size">{file_size_mb} MB</span>
        </li>
        """
    
    # Close HTML tags
    html_content += """
            </ul>
        </div>
    </body>
    </html>
    """
    
    return html_content

# Save the generated HTML files locally
os.makedirs('output', exist_ok=True)

# Generate and save root index.html
root_html_content = generate_html("root", files_by_folder['root'])
with open("output/index.html", "w") as f:
    f.write(root_html_content)

# Generate and save HTML files for each folder
for folder, files in files_by_folder.items():
    if folder != "root":
        folder_path = os.path.join('output', folder)
        os.makedirs(folder_path, exist_ok=True)
        html_content = generate_html(folder, files)
        with open(os.path.join(folder_path, "index.html"), "w") as f:
            f.write(html_content)

print("HTML files generated successfully.")


HTML files generated successfully.
